In [1]:
import stripe

In [75]:
import stripe
import getpass
stripe.api_key = getpass.getpass()

········


# creating merchant / organizer account

In [76]:
org_account1 = stripe.Account.create(type='express', idempotency_key="organizer1")

AuthenticationError: You did not provide an API key. You need to provide your API key in the Authorization header, using Bearer auth (e.g. 'Authorization: Bearer YOUR_SECRET_KEY'). See https://stripe.com/docs/api#authentication for details, or we can help at https://support.stripe.com/.

In [77]:
org_account1.id

'acct_1Izboq2fDhsPOJd0'

In [78]:
org_account2 = stripe.Account.create(type='express', idempotency_key="organizer2")

AuthenticationError: You did not provide an API key. You need to provide your API key in the Authorization header, using Bearer auth (e.g. 'Authorization: Bearer YOUR_SECRET_KEY'). See https://stripe.com/docs/api#authentication for details, or we can help at https://support.stripe.com/.

# generating hosted onboarding url

In [79]:
link = stripe.AccountLink.create(type='account_onboarding', account=org_account1.id,
                                 return_url='https://localhost:8000/return_url',
                                 refresh_url='https://localhost:8000/refresh_url')

AuthenticationError: You did not provide an API key. You need to provide your API key in the Authorization header, using Bearer auth (e.g. 'Authorization: Bearer YOUR_SECRET_KEY'). See https://stripe.com/docs/api#authentication for details, or we can help at https://support.stripe.com/.

In [34]:
link.url

'https://connect.stripe.com/express/onboarding/Ai3FpxAxnD9P'

In [57]:
org_account_updated = stripe.Account.retrieve(org_account1.id)

making sure transfers are activated

In [43]:
org_account1_updated.capabilities.transfers

'active'

# creating partner dashboard SSO link

re-execute this cell to get fresh link

In [74]:
stripe.Account.create_login_link(org_account1.id)

InvalidRequestError: Request req_uIOAveB0fJvDDG: Cannot create a login link for an account that has not completed onboarding.

# pay-in time

In [48]:
session = stripe.checkout.Session.create(
    payment_method_types=['card'],
    line_items=[{
      'price_data': {
        'currency': 'usd',
        'product_data': {
          'name': 'T-shirt',
        },
        'unit_amount': 2000,
      },
      'quantity': 1,
    }],
    payment_intent_data={
        'application_fee_amount': 0,
        'transfer_data': {
          'destination': org_account1.id,
        },
    },
    mode='payment',
    success_url='https://localhost:8000/return_url',
    cancel_url='https://localhost:8000/return_url',
  )

In [50]:
session.id

'cs_test_a1aUO82nv7r3OPLdFMha6iXCJu75Utui63CxSGSr2neRATOlv6P8dH1MJk'

```
python -m SimpleHTTPServer 8090
```

open 
http://localhost:8090/checkout.html

now let's see merchant account

In [51]:
stripe.Account.create_login_link(org_account1.id)

<LoginLink login_link at 0x113065360> JSON: {
  "created": 1622876544,
  "object": "login_link",
  "url": "https://connect.stripe.com/express/d15jFBkA7dNi"
}

one more payment this time with transfer_group

In [52]:
session2 = stripe.checkout.Session.create(
    payment_method_types=['card'],
    line_items=[{
      'price_data': {
        'currency': 'usd',
        'product_data': {
          'name': 'T-shirt',
        },
        'unit_amount': 2500,
      },
      'quantity': 1,
    }],
    payment_intent_data={
        'application_fee_amount': 0,
        'transfer_data': {
          'destination': org_account1.id,
        },
        'transfer_group': 'event_1'
    },
    mode='payment',
    success_url='https://localhost:8000/return_url',
    cancel_url='https://localhost:8000/return_url',
  )

In [54]:
session2.id

'cs_test_a1HfbMu3nRcGJexzpQRTb0Q4SZEJnCyG0gb4QM4DmVkP3PPRqwxnfMN4e3'

http://localhost:8090/checkout.html

after that balance should be 45... but no indication of transfer_group

# payout time

let's see a balance object. if there are any money available for transfer

pending looks like this, this we can't payout yet.
```
"pending": [
    {
      "amount": 4500,
      "currency": "usd",
      "source_types": {
        "card": 4500
      }
    }
```

In [66]:
stripe.Balance.retrieve(
  stripe_account=org_account1.id
)

<Balance balance at 0x113403e50> JSON: {
  "available": [
    {
      "amount": 0,
      "currency": "usd",
      "source_types": {
        "card": 0
      }
    }
  ],
  "instant_available": [
    {
      "amount": 0,
      "currency": "usd",
      "source_types": {
        "card": 0
      }
    }
  ],
  "livemode": false,
  "object": "balance",
  "pending": [
    {
      "amount": 4500,
      "currency": "usd",
      "source_types": {
        "card": 4500
      }
    }
  ]
}

In [62]:
updated_account = stripe.Account.modify(
  org_account1.id,
  settings={
    'payouts': {
      'schedule': {
        'interval': 'manual',
      },
    },
  }
)

let's try to payout. it won't work if balance isn't available yet for payout.

In [64]:
payout = stripe.Payout.create(
  amount=1000,
  currency='usd',
  stripe_account=org_account1.id,
)

InvalidRequestError: Request req_yRw5NayIEWlBrA: You have insufficient funds in your Stripe account for this transfer. Your card balance is too low.  You can use the /v1/balance endpoint to view your Stripe balance (for more details, see stripe.com/docs/api#balance).

# how much money linkedin/ owns in fees?

In [60]:
stripe.Balance.retrieve()

<Balance balance at 0x112fbccc0> JSON: {
  "available": [
    {
      "amount": 0,
      "currency": "usd",
      "source_types": {
        "card": 0
      }
    }
  ],
  "connect_reserved": [
    {
      "amount": 0,
      "currency": "usd"
    }
  ],
  "livemode": false,
  "object": "balance",
  "pending": [
    {
      "amount": -191,
      "currency": "usd",
      "source_types": {
        "card": -191
      }
    }
  ]
}

# can we use random account to transfer money to.

trying to create checkout session for inactive account, can we do that.

In [67]:
session2 = stripe.checkout.Session.create(
    payment_method_types=['card'],
    line_items=[{
      'price_data': {
        'currency': 'usd',
        'product_data': {
          'name': 'T-shirt',
        },
        'unit_amount': 2200,
      },
      'quantity': 1,
    }],
    payment_intent_data={
        'application_fee_amount': 0,
        'transfer_data': {
          'destination': org_account2.id,
        },
        'transfer_group': 'event_2'
    },
    mode='payment',
    success_url='https://localhost:8000/return_url',
    cancel_url='https://localhost:8000/return_url',
  )

InvalidRequestError: Request req_RbGHPuo2Wany3V: Your destination account needs to have at least one of the following capabilities enabled: transfers, legacy_payments